In [1]:
import h5py
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import optimizers
import keras
import pandas
import utils

Using TensorFlow backend.


In [2]:
train_folder = 'data/train'
test_folder = 'data/test'
validation_folder = 'data/validation'
labels_csv = 'train_labels.csv'
label_names = ['0', '1']
IMG_WIDTH = 450
IMG_HEIGHT = 450
batch_size = 8

In [3]:
utils.create_labelled_folders(train_folder, labels_csv, label_names)        
utils.create_validation_subfolders(train_folder, validation_folder, label_names, 0.2)        

Folders are already labelled


In [4]:
train_datagen = ImageDataGenerator(
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    rescale=1./255.,
    vertical_flip=True,
    horizontal_flip=True,
    rotation_range=90
)

validation_datagen = ImageDataGenerator(
    rescale=1./255.
)

train_generator = train_datagen.flow_from_directory(
    train_folder,
    batch_size=batch_size,
    target_size = (IMG_WIDTH, IMG_HEIGHT)
)

validation_generator = validation_datagen.flow_from_directory(
    validation_folder,
    batch_size=batch_size,
    target_size = (IMG_WIDTH, IMG_HEIGHT)
)

Found 1837 images belonging to 2 classes.
Found 458 images belonging to 2 classes.


In [ ]:
base_model = keras.applications.vgg16.VGG16(weights='imagenet', 
                                            include_top=False, 
                                            input_shape=(IMG_WIDTH, IMG_HEIGHT, 3), 
                                            classes=2)

In [ ]:
base_model.summary()

In [ ]:
x = base_model.output
x = (Flatten(input_shape=base_model.output_shape[1:], name="finetune"))(x)
x = (Dense(256, activation='relu'))(x)
x = (Dropout(0.5))(x)
x = (Dense(2, activation='sigmoid'))(x)

model = Model(inputs=base_model.inputs, outputs=x)

for layer in model.layers:
    if layer.name == "block5_conv3":
        break
    
    layer.trainable = False

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [5]:
model = keras.models.load_model('temp.h5')

In [6]:
num_training_samples = train_generator.n
num_validation_samples = validation_generator.n

model.fit_generator(
    train_generator,
    steps_per_epoch=num_training_samples//batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=num_validation_samples//batch_size)

Epoch 1/50
229/229 [==============================] - 334s - loss: 0.1811 - acc: 0.9350 - val_loss: 0.1456 - val_acc: 0.9496
Epoch 2/50
229/229 [==============================] - 215s - loss: 0.1670 - acc: 0.9293 - val_loss: 0.1525 - val_acc: 0.9500
Epoch 3/50
229/229 [==============================] - 212s - loss: 0.1749 - acc: 0.9342 - val_loss: 0.1408 - val_acc: 0.9422
Epoch 4/50
229/229 [==============================] - 212s - loss: 0.1615 - acc: 0.9334 - val_loss: 0.1658 - val_acc: 0.9322
Epoch 5/50
229/229 [==============================] - 212s - loss: 0.1500 - acc: 0.9430 - val_loss: 0.1400 - val_acc: 0.9522
Epoch 6/50
229/229 [==============================] - 212s - loss: 0.1674 - acc: 0.9380 - val_loss: 0.1342 - val_acc: 0.9600
Epoch 7/50
229/229 [==============================] - 212s - loss: 0.1690 - acc: 0.9299 - val_loss: 0.1469 - val_acc: 0.9489
Epoch 8/50
229/229 [==============================] - 211s - loss: 0.1657 - acc: 0.9383 - val_loss: 0.1480 - val_acc: 0.9522


In [37]:
model.save('invasive_species.h5')

In [10]:
test_datagen = ImageDataGenerator(
    rescale=1./255.
)

test_generator = test_datagen.flow_from_directory(
    test_folder,
    batch_size=batch_size,
    target_size = (IMG_WIDTH, IMG_HEIGHT)
)

Found 1531 images belonging to 1 classes.


In [11]:
num_test_samples = test_generator.n
test_predictions = model.predict_generator(test_generator, num_test_samples//batch_size)

In [13]:
test_predictions[:20]

array([[  4.22737179e-10,   1.00000000e+00],
       [  4.58515040e-12,   1.00000000e+00],
       [  9.63363469e-01,   2.90614516e-02],
       [  7.25922585e-01,   3.29287767e-01],
       [  4.03726155e-08,   9.99999881e-01],
       [  9.11682665e-01,   8.85884538e-02],
       [  9.59909618e-01,   4.19592559e-02],
       [  8.00499667e-09,   1.00000000e+00],
       [  8.28474045e-01,   1.89637035e-01],
       [  7.22374737e-01,   2.96274900e-01],
       [  1.65485492e-08,   1.00000000e+00],
       [  1.21329986e-05,   9.99985218e-01],
       [  2.11506276e-07,   9.99999762e-01],
       [  9.08112696e-10,   1.00000000e+00],
       [  8.71309042e-01,   1.15887389e-01],
       [  9.79272425e-01,   2.07115673e-02],
       [  8.60504091e-01,   1.34673640e-01],
       [  4.83975671e-02,   9.48808670e-01],
       [  7.50365257e-01,   2.33482733e-01],
       [  1.61438706e-07,   9.99999881e-01]], dtype=float32)

In [30]:
invasive = test_predictions[:,1]
names = np.arange(1, len(invasive)+1, dtype=np.int32)
submission_array = np.stack((names, invasive), axis=1)

In [36]:
np.save('submission.npy', submission_array)